# Lab 9,10 Hough line transformation

1. Example https://docs.opencv.org/3.4/d9/db0/tutorial_hough_lines.html

2. https://www.youtube.com/watch?v=2CIxM7x-Clc video speed lane detection

3. https://www.youtube.com/watch?v=6q5_A5wOwDM original video speed detection

4. https://www.youtube.com/watch?v=G2VaJvNNp4k result video (expected)

# Даалгавар 1
Hough transform ашиглан замын тусгаарлах зураасыг дүрс бичлэгээс илрүүлэх

# Даалгавар 2
Машины хурд тодорхойлох

Хугацаа 3 долоо хоног

# YouTube download video

In [2]:
# import cv2
# def FrameCapture(path):
  
#     # Path to video file
#     vidObj = cv2.VideoCapture(path)
  
#     # Used as counter variable
#     count = 0
  
#     # checks whether frames were extracted
#     success = 1
  
#     for i in range(1,1000):
  
#         # vidObj object calls read
#         # function extract frames
#         success, image = vidObj.read()
  
#         # Saves the frames with frame-count
#         cv2.imwrite("frame%d.jpg" % count, image)
  
#         count += 1

# FrameCapture("../Video/data.mp4")

In [3]:
# import pytube
# url = 'https://www.youtube.com/watch?v=2CIxM7x-Clc'
# pytube.YouTube(url).streams.get_highest_resolution().download('../Video')

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math

import random

def regionOfInterest(image): 
    
    height = image.shape[0] 
    width = image.shape[1]
    polygons = np.array([[(int(width/6), height), (int(width/2.5), int(height/1.45)), (int(width/1.9), int(height/1.45)), (int(width/1.3), height), (int(800),650), (int(400),650),(int(width/6), height)]])

    # create zero array size of img
    zeroMask = np.zeros_like(image)

    # fill polygon with ones
    cv2.fillConvexPoly(zeroMask, polygons, 1)

    # mask our original image to create roi
    roi = cv2.bitwise_and(image, image, mask = zeroMask)
    
    return roi


def getLineCoordinates(frame, lines):
    height = int(frame.shape[0]/1.5)
    slope, yIntercept = lines[0], lines[1]

    # get 1st bottom y coordinate
    y1 = frame.shape[0]

    # get 2nd top y coordinate
    y2 = int(y1-110)

    # first x coordinate, solving for x in y = mx+b
    x1 = int((y1-yIntercept)/slope)

    # seceond x coordinate, solving for x in y = mx+b
    x2 = int((y2-yIntercept)/slope)

    return np.array([x1,y1,x2,y2])

def getLines(frame, lines):
    copyImage = frame.copy()
    leftLine, rightLine = [], []
    roiHeight = int(frame.shape[0]/1.5)
    lineFrame = np.zeros_like(frame)
    for line in lines: 
        x1, y1, x2, y2 = line[0]
        # calculate slope & y intercept
        lineData = np.polyfit((x1,x2), (y1,y2), 1)
        slope, yIntercept = round(lineData[0], 8), lineData[1]
        if slope < 0: 
            leftLine.append((slope, yIntercept))
        else:
            rightLine.append((slope, yIntercept))

    # convert back into [x1,y1,x2,y2] format
    if leftLine:
        leftLineAverage = np.average(leftLine, axis = 0)
        left = getLineCoordinates(frame, leftLineAverage)
        try:
            cv2.line(lineFrame, (left[0],left[1]), (left[2],left[3]), (255, 0, 0), 3)
        except Exception as e: 
            print('Error', e)
    if rightLine:
        rightLineAverage = np.average(rightLine, axis = 0) 
        right = getLineCoordinates(frame, rightLineAverage)
        try:
            cv2.line(lineFrame, (right[0],right[1]), (right[2],right[3]), (255, 0, 0), 3)
        except Exception as e: 
            print('Error:', e)

    # return copyImage with lines weighted for transparency
    return cv2.addWeighted(copyImage, 0.8, lineFrame, 0.8, 0.0)

def region(image):
    height, width = image.shape
    triangle = np.array([[(0, height-100), (475, 375), (725, 400), (width-125, height-100)]])
    #triangle = np.array([[(int(width/6), height), (int(width/2.5), int(height/1.45)), (int(width/1.9), int(height/1.45)), (int(width/1.3), height)]])
    mask = np.zeros_like(image)
    
    mask = cv2.fillPoly(mask, triangle, 255)
    mask = cv2.bitwise_and(image, mask)
    return mask



def average(image, lines):
    left = []
    right = []

    if lines is not None:
      for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        #fit line to points, return slope and y-int
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        
        slope = parameters[0]
        y_int = parameters[1]
        #lines on the right have positive slope, and lines on the left have neg slope
        if slope < 0:
            left.append((slope, y_int))
        else:
            right.append((slope, y_int))
            
    #takes average among all the columns (column0: slope, column1: y_int)
    right_avg = np.average(right, axis=0)
    left_avg = np.average(left, axis=0)
    #create lines based on averages calculates
    left_line = make_points(image, left_avg)
    right_line = make_points(image, right_avg)
    return np.array([left_line, right_line])

def make_points(image, average):
    
    slope, y_int = average
    y1 = image.shape[0]
    #how long we want our lines to be --> 3/5 the size of the image
    y2 = int(y1 * (3/5))
    #determine algebraically
    x1 = int((y1 - y_int) // slope)
    x2 = int((y2 - y_int) // slope)
    return np.array([x1, y1, x2, y2])

def display_lines(image, lines):
    lines_image = np.zeros_like(image)
    #make sure array isn't empty
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line
            #draw lines on a black image
            cv2.line(lines_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return lines_image


cap = cv2.VideoCapture("../Video/data.mp4")
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('output2.mp4', fourcc, 20.0, (640,360))
output = cv2.VideoWriter(
        "output.avi", cv2.VideoWriter_fourcc(*'MPEG'), 
      30, (1080, 1920))
while(True):
    ret, frame = cap.read()
    if(ret):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        blur = cv2.GaussianBlur(gray, (5, 5), 0)

        edges = cv2.Canny(frame,100,200)

        copy = np.copy(frame)
        isolated = region(edges)
        cv2.imshow("p",isolated)
        #linesP = cv2.HoughLinesP(isolated, rho=2, theta=np.pi/180, threshold=150, lines=np.array([]), minLineLength=40, maxLineGap=3)
        linesP = cv2.HoughLinesP(isolated, rho=1, theta=np.pi/180, threshold=20, lines = np.array([]), minLineLength=10, maxLineGap=180)
        if linesP is not None:
            for i in range(0, len(linesP)):
                l = linesP[i][0]
                cv2.line(copy, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv2.LINE_AA)
        # averaged_lines = average(isolated, linesP)
        # black_lines = display_lines(copy, averaged_lines)
        # lanes = cv2.addWeighted(frame, 0.8, black_lines, 1, 1)
        lanes = getLines(frame, linesP)
        # writing the new frame in output
        out.write(lanes)
        cv2.imshow("output", lanes)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break
    else:
        break

cv2.destroyAllWindows()
output.release()
cap.release()

[ WARN:0@0.632] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@0.632] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin0 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@0.633] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@0.633] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_16645483

: 

In [ ]:
# import cv2
# def FrameCapture(path):
  
#     # Path to video file
#     vidObj = cv2.VideoCapture(path)
  
#     # Used as counter variable
#     count = 0
  
#     # checks whether frames were extracted
#     success = 1
  
#     for i in range(1,1000):
  
#         # vidObj object calls read
#         # function extract frames
#         success, image = vidObj.read()
  
#         # Saves the frames with frame-count
#         cv2.imwrite("frame%d.jpg" % count, image)
  
#         count += 1

# FrameCapture("../Video/data.mp4")


In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

def regionOfInterest(image): 
    
    height = image.shape[0] 
    width = image.shape[1]
    polygons = np.array([[(150, height-100), (475, 225), (775, 225), (width, height-100)]])

    # create zero array size of img
    zeroMask = np.zeros_like(image)

    # fill polygon with ones
    cv2.fillConvexPoly(zeroMask, polygons, 1)

    # mask our original image to create roi
    roi = cv2.bitwise_and(image, image, mask = zeroMask)
    
    return roi

def getLineCoordinates(frame, lines):
    height = int(frame.shape[0]/1.5)
    slope, yIntercept = lines[0], lines[1]

    # get 1st bottom y coordinate
    y1 = frame.shape[0]

    # get 2nd top y coordinate
    y2 = int(y1-110)

    # first x coordinate, solving for x in y = mx+b
    x1 = int((y1-yIntercept)/slope)

    # seceond x coordinate, solving for x in y = mx+b
    x2 = int((y2-yIntercept)/slope)

    return np.array([x1,y1,x2,y2])


# average out lines from hough transformation
def getLines(frame, lines):
    copyImage = frame.copy()
    leftLine, rightLine = [], []
    roiHeight = int(frame.shape[0]/1.5)
    lineFrame = np.zeros_like(frame)
    for line in lines: 
        x1, y1, x2, y2 = line[0]
        # calculate slope & y intercept
        lineData = np.polyfit((x1,x2), (y1,y2), 1)
        slope, yIntercept = round(lineData[0], 8), lineData[1]
        if slope < 0: 
            leftLine.append((slope, yIntercept))
        else:
            rightLine.append((slope, yIntercept))

    # convert back into [x1,y1,x2,y2] format
    if leftLine:
        leftLineAverage = np.average(leftLine, axis = 0)
        left = getLineCoordinates(frame, leftLineAverage)
        try:
            cv2.line(lineFrame, (left[0],left[1]), (left[2],left[3]), (255, 0, 0), 2)
        except Exception as e: 
            print('Error', e)
    if rightLine:
        rightLineAverage = np.average(rightLine, axis = 0) 
        right = getLineCoordinates(frame, rightLineAverage)
        try:
            cv2.line(lineFrame, (right[0],right[1]), (right[2],right[3]), (255, 0, 0), 2)
        except Exception as e: 
            print('Error:', e)

    # return copyImage with lines weighted for transparency
    return cv2.addWeighted(copyImage, 0.8, lineFrame, 0.8, 0.0)

# save finalized video
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('../Video/output.mp4', fourcc, 20.0, (640,360))

vid = cv2.VideoCapture('../Video/data.mp4')
count = 0
while (vid.isOpened()):
    ret,frame = vid.read()
    cv2.imshow("original frame", frame)

    # 1. convert to grayscale
    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 2. gaussian blur the grayscale frame to reduce noise, kernal size must be positive & odd
    kernel_size = 1
    gaussianFrame = cv2.GaussianBlur(grayFrame,(kernel_size,kernel_size),kernel_size)
    cv2.imshow('gaussianFrame', gaussianFrame)

    # 3. detect edges using canny edge detection
    low_threshold = 75
    high_threshold = 160
    edgeFrame = cv2.Canny(gaussianFrame, low_threshold, high_threshold)

    # 4. define region of interest
    roiFrame = regionOfInterest(edgeFrame)

    # 5. detect lines using hough's algorithm
    lines = cv2.HoughLinesP(roiFrame, rho=1, theta=np.pi/180, threshold=20, lines = np.array([]), minLineLength=10, maxLineGap=180)

    # 6. draw lines onto frame
    # version 1: without averaging lines
    # copyImage = frame.copy()
    # for line in lines: 
    #     x1, y1, x2, y2 = line[0]
    #     cv2.line(copyImage, (x1,y1), (x2,y2), (255, 0, 0), 2)
    # cv2.imshow("final", copyImage)

    # version 2: with averaging lines
    imageWithLines = getLines(frame, lines)
    cv2.imshow("final", imageWithLines)

    # Optional: if you want to save individual final frames
    # cv2.imwrite("frame%d.jpg" % count, imageWithLines)     # save frame as JPEG file  
    # count += 1

    # save frame final video output
    out.write(imageWithLines)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

vid.release()
out.release()
cv2.destroyAllWindows()


(<unknown>:54520): GStreamer-CRITICAL **: 15:18:53.154: gst_element_make_from_uri: assertion 'gst_uri_is_valid (uri)' failed
[ WARN:0@0.588] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2180) open OpenCV | GStreamer warning: cannot link elements
[ WARN:0@0.622] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@0.622] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin0 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0

: 

: 